## Exercise 2


In [1]:
import math

In [2]:
def RiceEncode(value, k):
    m = math.pow(2, k)
    
    quotient =  int(math.floor(value/m))
    remainder = int(value % m)
    
    quotientCode = ('1' * quotient) + '0'
    remainderCode = format(remainder, f'0{k}b')
    
    encodedValue = quotientCode + remainderCode
    
    #print(f'value: {value}, {bin(value)[2:]}, k: {k}')
    #print(f'quotient: {quotient}, remainder: {remainder}')
    #print(f'quotient_code: {quotientCode}, remainder_code: {remainderCode}')
    #print(f'encoded_value: {encodedValue}')
    return int(encodedValue, 2)

def RiceDecode(value, k):
    m = math.pow(2, k)
    strValue = bin(value)[2:]
    indexOfFirstZero = str(strValue).index('0')
    print(f'index of first zero: {indexOfFirstZero}')
    
    quotientCode = strValue[0:indexOfFirstZero]
    remainderCode = strValue[indexOfFirstZero + 1:]
    print(f'quotient code: {quotientCode}0, remainder code: {remainderCode}')
    
    quotient = len(quotientCode)
    remainder = int(remainderCode, 2)
    print(f'quotient: {quotient}, remainder: {remainder}')
    
    decodedValue = int(quotient * m + remainder)
    return decodedValue


def EncodeFileWithRiceEncode(source, destination, k, bufSize):
    #Read from source byte by byte.
    #Encode each byte. The encoded value can be longer or shorter than a byte.
    #If it is longer, split it in chunks one-byte long and add it to the buffer
    #If it is shorter, packs it into one byte and fill the remaining bits with the next chunk
    
    #buffer
    buffer = bytearray()
    #temporary byte where to store bits
    dByte = 0
    #bits left empty in the dByte
    bitsLeft = 8
    
    with open(source, "rb") as sStream:
        with open(destination, "wb") as dStream:
            #read from source byte-by-byte
            while (sByte := sStream.read(1)):
                #print(f'sByte: {int.from_bytes(sByte, "big")}')
                #Encode the byte
                iEncByte = RiceEncode(int.from_bytes(sByte, "big"), k);
                sEncByte = bin(iEncByte)[2:]
                #print(f'sByte: { int.from_bytes(sByte, "big")}, iEncByte: { iEncByte}, sEncByte: { sEncByte}')
                #index store the current position in the encoded value
                index = 0
                #pack the encode value in chunks into dByte
                while(index < len(sEncByte)):
                    #read bytes from index to nextIndex, max 8 bits, not going beyong the end of encoded value
                    nextIndex = min(index+bitsLeft, len(sEncByte))
                    #how many bits left empty in dByte
                    bitsLeft -= (nextIndex - index)
                    #pack the bits into dByte
                    #If the number of bits to pack fills dByte, there is no bit shift to do
                    #If the number of bits to pack is shorter than the number of bits left, 
                    #shift the bits to the left so as to pack them into the dByte and left the free space on the right side
                    dByte |= (int(sEncByte[index:nextIndex], 2) << bitsLeft)
                    
                    #If there are no bits left, meaning that the dByte is fully packed
                    if(bitsLeft == 0):
                        #add the dByte to the buffer
                        buffer.append(dByte)
                        #empty the dByte for next iteration
                        dByte = 0
                        #reset the number of available bits in dByte
                        bitsLeft = 8
                        #Check the buffer, if its full (its size is greater than or equal to the buffers size set in bufSize)
                        if(len(buffer) >= bufSize):
                            #write the buffer into the destination stream
                            dStream.write(buffer)
                            #empty the buffer
                            buffer = bytearray()
                    
                    #Update the index for the next iteration throught the encoded value
                    index = nextIndex
                
            #If sStream is EOF but there is a dByte partially filled, add it to the buffer and flush
            if(bitsLeft != 8):
                buffer.append(dByte)
                
            dStream.write(buffer)
    
                    

In [3]:
k = 4
value = 82

encodedValue = RiceEncode(int(value), k)
print(f'value: {value}, encoded value: {encodedValue}')

decodedValue = RiceDecode(encodedValue, k)
print(f'decoded value: { decodedValue}')

value: 82, encoded value: 994
index of first zero: 5
quotient code: 111110, remainder code: 0010
quotient: 5, remainder: 2
decoded value: 82


In [8]:
originalFile = "Exercise2_Files\Sound1.wav"
encodedFile = "Exercise2_Files\Sound1_Enc.ex2"
decodedFile = "Exercise2_Files\Sound1_Enc_Dec.wav"

k = 4
bufSize = 256

EncodeFileWithRiceEncode(originalFile, encodedFile, k, bufSize)

print("Done!")


Done!
